In [2]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/tatabahasabm.tripod.com/quiz-tatabahasa.jsonl

In [3]:
from tqdm import tqdm
import os
import json

In [8]:
instructions = []
with open('quiz-tatabahasa.jsonl') as fopen:
    for no, l in enumerate(fopen):
        l = json.loads(l)
        soalan = [l['question']]
        jawapan = None
        for c, k in l['choices'].items():
            soalan.append(f"{c}. {k['text']}")
            if k['answer']:
                jawapan = c
        
        i = 'Jawab soalan yang diberikan' if l['instruction'] is None else l['instruction']
        q = '\n'.join(soalan)
        s = f"{i}\n{q}\nJawapan: {jawapan}"
        instructions.append(s)
        
len(instructions)

349

In [10]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [11]:
!mkdir tatabahasa

In [13]:
from tqdm import tqdm
import os

for i in tqdm(range(0, len(instructions), 1)):
    filename = f'tatabahasa/{i}.json'
    if os.path.exists(filename):
        continue
    try:
        message_text = [
            {"role":"user","content": f"{instructions[i]}\n\ngenerate similar questions included answers like above."},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        with open(filename, 'w') as fopen:
            json.dump(completion.choices[0]['message']['content'], fopen)
        
    except Exception as e:
        print(e)

 48%|███████████████████▊                     | 169/349 [26:00<24:05,  8.03s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 62%|█████████████████████████▍               | 217/349 [32:50<15:33,  7.07s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 76%|███████████████████████████████▏         | 265/349 [40:02<13:39,  9.75s/it]

'content'


 95%|██████████████████████████████████████▉  | 331/349 [50:17<02:35,  8.61s/it]

'content'


100%|█████████████████████████████████████████| 349/349 [52:58<00:00,  9.11s/it]


In [14]:
augmentation = []
for i in tqdm(range(0, len(instructions), 1)):
    filename = f'tatabahasa/{i}.json'
    try:
        with open(filename) as fopen:
            data = json.load(fopen)
        splitted = data.split('\n\n')
        for s in splitted:
            a = s.split('Jawapan:')[1].strip()
            augmentation.append({
                'question': s.split('Jawapan:')[0].strip(),
                'answer': a,
                
            })
    except:
        pass

100%|██████████████████████████████████████| 349/349 [00:00<00:00, 59475.54it/s]


In [15]:
len(augmentation)

2005

In [17]:
augmentation[-1]

{'question': '5. Dia memberi ........ supaya tidak ........ rasa tidak puas hati.\nA. pandangan-pandangan, membangkitkan\nB. saranan-saranan, menimbulkan\nC. arahan-arahan, mencetuskan\nD. komen-komen, mengundang',
 'answer': 'B'}

In [18]:
with open('synthetic-tatabahasa.jsonl', 'w') as fopen:
    for a in augmentation:
        fopen.write(f'{json.dumps(a)}\n')